## Instalando Requisitos

### webdrivermanager
### https://pypi.org/project/webdriver-manager/

### versao Chromium e driver(chrome-win.zip e chromedriver_win32.zip): https://commondatastorage.googleapis.com/chromium-browser-snapshots/index.html?prefix=Win/991466/

## Imports

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import time

## Cria lista de nomes possíveis de cargos utilizando a base_salarios_completa.csv como fonte

In [ ]:
import pandas as pd
df = pd.read_csv('base_salarios_completa.csv')

# Removendo espaços do Código Cargo
df['Código Cargo'] = (
    df['Código Cargo'].str.replace('\n','')
    .str.rstrip()
    .str.lstrip()    
    .to_list()
)
# Filtra Matriculas válidas
df = df[(df['Matricula']!="true") & (df['Código Cargo']!="")] # Remove a linha true, erro da base
# Divide o Código cargo em numero e nome
df['Numero Cargo'] = df['Código Cargo'].str.split(' ').str[0] 
df['Numero Cargo'] = df['Numero Cargo'].astype(int)
df['Nome Cargo'] = (
    df['Código Cargo'].str.split(' ')
    .str[2:]
    .str.join(' ')
    .str.replace(' \n', '')
    .str.replace('\n', '')
)

df['Nome Cargo'] = (
    df['Nome Cargo'].str.replace('\n','')
    .str.rstrip()
    .str.lstrip()    
    .to_list()
)
cargos = df[~df['Nome Cargo'].isna()].drop_duplicates('Nome Cargo')['Nome Cargo'].to_list()
# Cria lista de possíveis nomes de cargos
cargos_v2 = list()
for x in cargos: cargos_v2 = cargos_v2 + x.replace(" ", ".").replace("/", ".").replace("(A)", "").replace("-", ".").split('.')

# Remove duplicidade
cargos_v2 = list(dict.fromkeys(cargos_v2))

# Remove espaço
cargos_v2.remove("")

# Tamanho cargos_v2 
len(cargos_v2)

# Adicionando alfabeto
cargos_v2 = cargos_v2 + ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

## Inicializar classe RemuneracoesCampinas com Selenium e Chromium

In [ ]:
class InformacoesDeCargo:
    def __init__(self):
        self.site_link = "https://cargosesalarios.campinas.sp.gov.br/index.php?op=6"
        self.options = Options()
        self.options.binary_location = f"{os.getcwd()}\\chrome-win\\chrome.exe"
        self.driver = webdriver.Chrome(executable_path = f"{os.getcwd()}\\chromedriver.exe", options = self.options)
                
    def abrir_site(self):
        time.sleep(2)
        self.driver.get(self.site_link)
        time.sleep(7)
        
    def get_driver(self): return self.driver

## Desenvolvimento do download

In [ ]:
ic = InformacoesDeCargo()
dv = ic.get_driver()
ic.abrir_site()

# Cria listas vazias
lista_codigo_cargo_ja_baixado = list()
lista_dicionario_dados_cargos = []
lista_pesquisas_realizadas = list()

#Altera para a aba principal
dv.switch_to.window(dv.window_handles[0])

for cargo in [x for x in cargos_v2 if x not in lista_pesquisas_realizadas]:
    print(cargo)
    form = dv.find_element("xpath", "/html/body/div[3]/div/div/div/div[2]/div/div/input")
    form.clear()
    form.send_keys(cargo)
    dv.find_element("xpath", "/html/body/div[3]/div/div/div/div[2]/div/div/div/button").click()
    time.sleep(0.5)
    try:
        if dv.find_element("xpath", "/html/body/div[3]/div/div/div/div[3]/div/div/ul/li/span").text == "Nenhum cargo com este nome!": 
            print("Sem resultado na pesquisa")
            lista_pesquisas_realizadas.append(cargo)
            continue
    except:
        print("Encontrou resultado")

    raw_resposta_cargos = dv.find_element("xpath", "/html/body/div[3]/div/div/div/div[3]/div/div[2]/table/tbody").text.split('\n')
    lista_codigo_cargo = list()
    lista_nome_cargo = list()

    for nome in raw_resposta_cargos:
        codigo_cargo = nome.split(' ', 1)[0]
        nome_cargo = nome.split(' ', 1)[0]
        lista_codigo_cargo.append(codigo_cargo)
        lista_nome_cargo.append(nome_cargo)
    lista_codigo_cargo


    for idx, codigo_cargo in enumerate([x for x in lista_codigo_cargo if x not in lista_codigo_cargo_ja_baixado]):
        # Obtem a URL do cargo a baixar
        url_cargo_a_baixar = dv.find_element("xpath", f"//*[text()='{codigo_cargo}']").find_element("xpath", "./..").find_element("xpath", "td[2]/a").get_attribute('href')

        # Abrir uma nova guia
        dv.execute_script("window.open('');")

        # Alternar para a nova guia
        dv.switch_to.window(dv.window_handles[-1])
        dv.get(url_cargo_a_baixar)

        # Coleta os dados do cargo, com alguns tratamentos previos
        dados_cargo = {
                'codigo_cargo': codigo_cargo,
                'nome_completo': dv.find_element("xpath", "/html/body/div[3]/div/div/div[2]/div/ul/li[1]").text.replace("Cargo: ", "").replace("\nCARREIRA", ""),
                'jornada_mensal': dv.find_element("xpath", "/html/body/div[3]/div/div/div[2]/div/ul/li[2]").text.replace("Jornada Mensal: ", ""),
                'requisitos': dv.find_element("xpath", "/html/body/div[3]/div/div/div[2]/div/ul/li[3]").text.replace("Requisitos: ", ""),
                'experiencia': dv.find_element("xpath", "/html/body/div[3]/div/div/div[2]/div/ul/li[4]").text.replace("Experiência: ", ""),
                'atribuicoes': dv.find_element("xpath", "/html/body/div[3]/div/div/div[2]/div/ul/li[5]").text.replace("Atribuições:\n ", ""),
                'grupo': dv.find_element("xpath", "/html/body/div[3]/div/div/div[2]/div/ul/li[6]").text.replace("Grupo: ", ""),
                'lei': dv.find_element("xpath", "/html/body/div[3]/div/div/div[2]/div/ul/li[7]").text.replace("Lei: ", ""),
                'decreto': dv.find_element("xpath", "/html/body/div[3]/div/div/div[2]/div/ul/li[8]").text.replace("Decreto: ", ""),
            }

        # Adiciona dados do cargo a lista
        lista_dicionario_dados_cargos.append(dados_cargo)
        
        # Marca que ja foi baixado
        lista_codigo_cargo_ja_baixado.append(codigo_cargo)
          
        # Imprime feedback
        print(f"Lista de códigos de cargos ja baixado : {lista_codigo_cargo_ja_baixado}")
        print(f"Dados Cargo : {dados_cargo}")

        # Fecha a aba
        dv.close()

        # Alternar de volta para a guia original
        dv.switch_to.window(dv.window_handles[0])

    # Marca que a pesquisa ja foi realizada
    lista_pesquisas_realizadas.append(cargo)

In [ ]:
len(lista_dicionario_dados_cargos) # 543

In [ ]:
# cria e salva o dataframe
pd_dados_cargos = pd.DataFrame(lista_dicionario_dados_cargos)
pd_dados_cargos.to_csv("dados_cargos.csv")